In [1]:
from data import get_TT_labeled_dataloaders

In [2]:
import data_config
import pandas as pd
gex_features_df = pd.read_csv(data_config.gex_feature_file, index_col=0)


In [3]:
labeled_tcga_dataloader = get_TT_labeled_dataloaders(
        gex_features_df=gex_features_df,
        tcga_cancer_type = 'MESO',
        batch_size = 1
)

In [5]:
labeled_tcga_dataloader.dataset.tensors

(tensor([[ 0.5407,  0.5868, -0.6704,  ..., -0.2290,  1.0653,  0.2842],
         [ 1.8017,  0.5771, -0.9504,  ...,  0.1262, -0.0922, -0.0711],
         [ 1.3036,  1.1785, -1.2443,  ...,  0.2238,  0.0511,  0.7721],
         ...,
         [ 1.5439,  0.4805, -1.0148,  ...,  1.3495, -0.0509,  0.1098],
         [ 0.2442,  1.9082, -1.0839,  ...,  0.5633,  0.2527,  1.4879],
         [ 1.8961,  1.3503, -0.6079,  ...,  1.2199,  1.2754,  0.7294]]),
 tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0], dtype=torch.int32))

In [4]:
tcga_gex_feature_df = gex_features_df.loc[gex_features_df.index.str.startswith('TCGA')]
tcga_gex_feature_df.index = tcga_gex_feature_df.index.map(lambda x: x[:12])
tcga_gex_feature_df = tcga_gex_feature_df.groupby(level=0).mean()
tcga_treatment_df = pd.read_csv(data_config.tcga_first_treatment_file)
tcga_response_df = pd.read_csv(data_config.tcga_first_response_file)

tcga_treatment_df.drop_duplicates(subset=['bcr_patient_barcode'], keep=False, inplace=True)
tcga_treatment_df.set_index('bcr_patient_barcode', inplace=True)

tcga_response_df.drop_duplicates(inplace=True)
tcga_response_df.drop_duplicates(subset=['bcr_patient_barcode'], inplace=True)
tcga_response_df.set_index('bcr_patient_barcode', inplace=True)

In [5]:
tcga_treatment_df['tcga_project'].value_counts()

BRCA    209
GBM     133
LGG     129
CESC    106
HNSC    101
SKCM     87
STAD     74
PAAD     69
KIRC     51
PRAD     37
THCA     31
LIHC     30
COAD     29
UCEC     29
READ     28
LUAD     23
MESO     22
LUSC     20
SARC     17
ESCA     13
KIRP     11
BLCA     11
OV        9
TGCT      6
UCS       5
UVM       4
CHOL      4
THYM      4
KICH      3
PCPG      2
ACC       2
DLBC      1
Name: tcga_project, dtype: int64

In [6]:
tcga_drug_barcodes = tcga_treatment_df.index
drug_tcga_response_df = tcga_response_df.loc[tcga_drug_barcodes.intersection(tcga_response_df.index)]
labeled_tcga_gex_feature_df = tcga_gex_feature_df.loc[
        drug_tcga_response_df.index.intersection(tcga_gex_feature_df.index)]
labeled_df = tcga_response_df.loc[labeled_tcga_gex_feature_df.index]

all(labeled_df.index == labeled_tcga_gex_feature_df.index)

True

In [7]:
labeled_df['tcga_project'].value_counts()

SKCM    55
KIRC    39
LGG     36
HNSC    31
PAAD    31
STAD    28
LIHC    27
CESC    18
BRCA    15
LUAD    14
PRAD    12
SARC    12
GBM     12
MESO    11
LUSC    11
COAD    10
BLCA     7
ESCA     6
KIRP     5
KICH     3
READ     3
UCEC     3
CHOL     3
OV       2
PCPG     2
UCS      2
ACC      2
THCA     1
UVM      1
THYM     1
Name: tcga_project, dtype: int64

In [8]:
#全癌种药物
tcga_drug_barcodes = tcga_treatment_df.index
drug_tcga_response_df = tcga_response_df.loc[tcga_drug_barcodes.intersection(tcga_response_df.index)]
labeled_tcga_gex_feature_df = tcga_gex_feature_df.loc[
        drug_tcga_response_df.index.intersection(tcga_gex_feature_df.index)]
labeled_df = tcga_response_df.loc[labeled_tcga_gex_feature_df.index]
labeled_df['drug'] = tcga_treatment_df.loc[labeled_df.index,'pharmaceutical_therapy_drug_name']

labeled_df['drug'].value_counts()

Temozolomide    46
Gemcitabine     46
Cisplatin       40
NOS             35
Sorafenib       26
                ..
Etoposide        1
Oxaliplatin      1
Degarelix        1
Methotrexate     1
Pemetrexed       1
Name: drug, Length: 78, dtype: int64

In [10]:
print(labeled_df['drug'].value_counts()[0:20])

Temozolomide    46
Gemcitabine     46
Cisplatin       40
NOS             35
Sorafenib       26
Fluorouracil    21
Sunitinib       12
Capecitabine    12
Carboplatin     10
Dacarbazine      9
Anastrozole      9
Leuprolide       9
Doxorubicin      8
Aldesleukin      7
Erlotinib        6
Interferon A     5
Docetaxel        4
Paclitaxel       4
Tamoxifen        4
Ipilimumab       4
Name: drug, dtype: int64


In [11]:
labeled_df.to_csv('output/label_all_cancers.csv')

In [22]:
#选癌种
tcga_drug_barcodes = tcga_treatment_df.index[tcga_treatment_df['tcga_project'] == 'SKCM']
drug_tcga_response_df = tcga_response_df.loc[tcga_drug_barcodes.intersection(tcga_response_df.index)]
labeled_tcga_gex_feature_df = tcga_gex_feature_df.loc[
        drug_tcga_response_df.index.intersection(tcga_gex_feature_df.index)]
labeled_df = tcga_response_df.loc[labeled_tcga_gex_feature_df.index]
labeled_df['drug'] = tcga_treatment_df.loc[labeled_df.index,'pharmaceutical_therapy_drug_name']

all(labeled_df.index == labeled_tcga_gex_feature_df.index)

True

In [23]:
labeled_df['drug'].value_counts()

NOS                            14
Dacarbazine                     9
Interferon A                    4
Vemurafenib                     4
Ipilimumab                      4
Temozolomide                    3
Fotemustine                     3
Aldesleukin                     2
Pembrolizumab                   1
PV-10                           1
Sargramostim                    1
MAGE-A3                         1
PNU-159548                      1
Polyvalent Melanoma Vaccine     1
Vemurafenib                     1
PI-88                           1
Melphalan                       1
Peginterferon alfa-2a           1
Diphencyprone                   1
Nivolumab                       1
Name: drug, dtype: int64

In [ ]:


    if days_threshold is None:
        days_threshold = np.median(labeled_df.days_to_new_tumor_event_after_initial_treatment)

    drug_label = np.array(labeled_df.days_to_new_tumor_event_after_initial_treatment > days_threshold, dtype='int32')
    # # #加上index
    # tcga_label_df = pd.DataFrame(index = labeled_df.index,data = drug_label,columns=['label'])
    # tcga_label_df.to_csv(f'label_index/{drug}_index.csv')

    labeled_tcga_dateset = TensorDataset(
        torch.from_numpy(labeled_tcga_gex_feature_df.values.astype('float32')),
        torch.from_numpy(drug_label))

    batch_size = 1
    labeled_tcga_dataloader = DataLoader(labeled_tcga_dateset,
                                         batch_size=batch_size,
                                         shuffle=False) #保持Patient原本的顺